In [6]:
# Header block to include all modules that must be imported ahead of time
# Only needs to be run once per session, and each time a new module is added

import requests
import json
import pandas as pd # this will need to be installed via command line first
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [7]:
# Method to get all the revelent player attributes for every player currently signed on every NFL team
# The CBS Sports website is used as the source

def GetPlayterAttributes(team: list): # accepts a list ['team-name', 'LOC']
    # Use requests to get the raw HTML response from cbssports.com
    CBSResponse = requests.get(f'https://www.cbssports.com/nfl/teams/{team[1]}/{team[0]}/roster/')
    
    # If we get a normal response proceed with data scraping
    if CBSResponse.status_code == 200:
                 
        # Confirm the request was successful
        print(f"Request succeeded with status code {CBSResponse.status_code}")
        
        # Use Pandas to read the HTML content and put it into a DataFrame
        TableTag = 'TableBase-table'
        DataFrames = pd.read_html(CBSResponse.content, attrs={'class': TableTag})
        
        # Select the relevant data from the DataFrame(offense, defense, and special teams)
        OffenseDF = DataFrames[0][['Player', 'POS', 'HT', 'WT', 'DOB (AGE)', 'EXP']]
        DefenseDF = DataFrames[1][['Player', 'POS', 'HT', 'WT', 'DOB (AGE)', 'EXP']]
        SpecialDF = DataFrames[2][['Player', 'POS', 'HT', 'WT', 'DOB (AGE)', 'EXP']]
        
        # Combine the three DataFrames into one
        FullTeamDF = pd.concat([OffenseDF, DefenseDF, SpecialDF], axis = 0, ignore_index = True)
        
        # Rename the DOB coloumn
        FullTeamDF = FullTeamDF.rename(columns = {'DOB (AGE)' : 'DOB'})
        
        # Create a new column for player age to separate it from DOB
        AgeList = list(FullTeamDF['DOB'].str.slice(-3, -1))
        # Convert the items in the list to integers
        AgeList = [int(s) if s else 22 for s in AgeList] # account for player with no age data
        # Create the new column for player age
        FullTeamDF['AGE'] = AgeList
        # replace the DOB coloumn with just the DOB
        FullTeamDF['DOB'] = FullTeamDF['DOB'].str.split(expand = True)[0]
        
        # Method to clean the player names since CBS puts player news directly in the name cell
        def CleanNameCell(NameText):
            # Split the name cell into a list
            NameText = NameText.split(' ')
            
            # If the name cell doesn't contain player news and no name postfix
            if len(NameText) == 3:
                # Remove the first initial by selecting the rest of the elements
                NameText = NameText[1:]
                # Extract the first name from the first-last name string
                NameText[0] = NameText[0][len(NameText[1]):]
                # Create new string with first and last name
                NameText = ' '.join(NameText)
                
            # The name cell contains a name postfix
            elif len(NameText) == 5:
                # Create new list to store the frist name, last name, postfix
                NameTextPostfix = []
                # Extract the first name from the first-suffix name string, add it to the list
                NameTextPostfix.append(NameText[2][len(NameText[4]):])
                # Add the last name to the list
                NameTextPostfix.append(NameText[1])
                # Add the postfix to the list
                NameTextPostfix.append(NameText[4])
                # Turn the list into a string
                NameText = ' '.join(NameTextPostfix)

            # Otherwise the player name cell contains player news, injury status
            else:
                # Select only the capitilized words from the player cell
                NameTextCapitalized = [s for s in NameText if s and not s[0].islower()]
                # Select the first inital which is always the first item in NameText when there is news, injury status
                First = NameTextCapitalized[0][0]
                # Select the last which is always the second item in NameText when there is news, injury status
                LastName =  NameTextCapitalized[1]
                # Create postfix varaible in case player name has it
                Postfix = ''
                PostfixList = {'Jr.', 'II','III', 'IV'} # set of possible postfixes

                # Iterate through the items in NameTextCapitalzied from the third item to the end
                for s in range(3, (len(NameTextCapitalized) - 1)):
                    # Make sure we're always comparing to the first letter of current item
                    FirstTemp = NameTextCapitalized[s][0]
                    # If the current element first char is the first initial and the next element is last name
                    if NameTextCapitalized[s][0] == FirstTemp and NameTextCapitalized[s + 1].replace(":", "") == LastName:
                        # If we haven't reached the second to last elementand the next next element is a postfix
                        if NameTextCapitalized[s + 2] in PostfixList:
                            # Set the postfix variable to the next next element
                            Postfix = NameTextCapitalized[s + 2].replace(".", "") # remove the period from the postfix

                        # Set the first and last name 
                        First = NameTextCapitalized[s]
                        LastName = NameTextCapitalized[s + 1].replace(":", "") # remove the possible colon from the last name
                        # No break point because the first instance of the full name might not contain a postfix
                        
                        
                # Recreats NameText with the first name, last name
                NameText = [First, LastName]
                # If a postfix was found, add it to the list
                if Postfix:
                    NameText.append(Postfix)
                
                NameText = ' '.join(NameText)

            
            return NameText
        

        # Apply the method to the player names column
        FullTeamDF['Player'] = FullTeamDF['Player'].apply(CleanNameCell)  
        # Sort the DataFrame by player name
        FullTeamDF = FullTeamDF.sort_values(by = ['Player'], ignore_index = True) 
        
        # Add the team location to make it easier to match players up in different files
        FullTeamDF['TeamLOC'] = team[1]
        # Create new column order and apply it
        new_columns = ['Player', 'TeamLOC', 'POS', 'HT', 'WT', 'DOB', 'AGE', 'EXP']
        FullTeamDF = FullTeamDF.loc[:,  new_columns]
        # Rename columns for more consistency
        FullTeamDF.columns = ['Name', 'Team', 'POS', 'HT', 'WT', 'DOB', 'AGE', 'EXP']
        
        # Capitilize the team name for consistency
        TeamNameCapitalized = (team[0].replace("-", " ")).split(' ')
        # We have to account the 49ers and their team name starting with a number
        TeamNameCapitalized = [s.capitalize() if not s[0].isdigit() else s for s in TeamNameCapitalized]
        TeamNameCapitalized = ' '.join(TeamNameCapitalized)
        
        # Create a .csv file with all the player names and their cap hits
        FullTeamDF.to_csv(f'./data/player-attributes-lists/{TeamNameCapitalized}-Player-Attribute-List.csv', index = False, header = False)
        
        # Read the csv back for testing purposes
        FullTeamCSV = pd.read_csv(f'./data/player-attributes-lists/{TeamNameCapitalized}-Player-Attribute-List.csv', header = None)
        FullTeamCSV.columns = ['Name', 'Team', 'POS', 'HT', 'WT', 'DOB', 'AGE', 'EXP']
        print(FullTeamCSV)
        
        return 0

    # If we don't get a normal reponse, stop scraping
    else:
        print(f"Request failed with status code {CBSResponse.status_code}")
        return -1



In [8]:
# Method to get the revelent attributes for every player currently signed on every NFL team
# Reads in the team list csv, and calls the GetTeamSalaryCapHits method for each team
def GetAllPlayerAttributes():
    # Read in the full team list csv
    FullTeamListCSV = pd.read_csv('./data/team-attribute-lists/Teams-List.csv', header = None)
    FullTeamListCSV.columns = ['TeamName', 'Abbreviation', 'Conference', 'Division']
    
    # Create a list of all the team names and abbreviations as a list of lists
    TeamList = []
    # Take the first two columns of each row, convert to a list, append to TeamList
    FullTeamListCSV.apply(lambda row: TeamList.append(row[:2].tolist()), axis = 1)
    
    # Get the salary cap hits for each team
    for team in TeamList:
        # format the team name to be used in the URL
        team[0] = team[0].replace(" ", "-").lower()
        GetPlayterAttributes(team)
    

GetAllPlayerAttributes()

Request succeeded with status code 200
                       Name Team  POS    HT   WT         DOB  AGE EXP
0              Aaron Brewer  ARI   LS   6-5  232    7/5/1990   34  13
1           Andre Baccellia  ARI   WR  5-10  175    1/7/1997   27   2
2            Andre Chachere  ARI   DB   6-0  197   2/10/1996   28   4
3          Austen Pleasants  ARI   OT   6-7  330   8/22/1997   26   1
4                BJ Ojulari  ARI   LB   6-2  248    4/5/2002   22   2
5                Ben Stille  ARI   DE   6-4  296  11/12/1997   26   2
6        Bernhard Seikovits  ARI   TE   6-5  262   7/24/1997   26   1
7             Bilal Nichols  ARI   DT   6-3  313   9/14/1996   27   7
8            Blake Gillikin  ARI    P   6-2  196   1/21/1998   26   5
9          Blake Whiteheart  ARI   TE   6-4  247   3/22/2000   24   2
10              Bobby Price  ARI   DB   6-3  208   4/25/1998   26   5
11              Budda Baker  ARI   SS  5-10  195   1/10/1996   28   8
12           Cameron Thomas  ARI   LB   6-4  267   

Request succeeded with status code 200
                       Name Team  POS    HT   WT         DOB  AGE EXP
0               Adisa Isaac  BAL   LB   6-4  254   10/4/2001   22   R
1            Andrew Vorhees  BAL   OG   6-6  310   1/21/1999   25   1
2      Ar'Darius Washington  BAL  SAF   5-8  176   11/2/1999   24   3
3             Arthur Maulet  BAL   CB   5-9  190   7/13/1993   31   8
4                Beau Brade  BAL   DB   6-0  203   1/29/2002   22   R
5             Ben Cleveland  BAL   OG   6-6  370   8/25/1998   25   4
6          Brandon Stephens  BAL   CB   6-1  215  12/29/1997   26   4
7               Brent Urban  BAL   DE   6-7  309    5/5/1991   33  11
8      Broderick Washington  BAL   DT   6-2  314   12/4/1996   27   5
9             C.J. Ravenell  BAL   DE   6-5  280           —   22   R
10            Charlie Kolar  BAL   TE   6-6  257   2/10/1999   25   3
11              Chris Board  BAL   LB   6-2  229   7/23/1995   28   7
12            Chris Collier  BAL   RB  5-11  203   

                    Name Team  POS    HT   WT         DOB  AGE EXP
0       A'Shawn Robinson  CAR   DT   6-3  320   3/21/1995   29   9
1           Adam Thielen  CAR   WR   6-2  195   8/22/1990   33  11
2              Alex Cook  CAR  SAF   6-1  196   5/21/1999   25   2
3            Amare Barno  CAR   LB   6-6  245   4/26/1999   25   3
4            Andrew Raym  CAR    C   6-4  327    5/2/2001   23   R
5            Andy Dalton  CAR   QB   6-2  220  10/29/1987   36  14
6         Austin Corbett  CAR    C   6-4  306    9/5/1995   28   7
7          Badara Traore  CAR   OT   6-7  320   3/12/1997   27   1
8      Brady Christensen  CAR   OG   6-6  300   9/27/1996   27   4
9            Bryce Young  CAR   QB  5-10  204   7/25/2001   22   2
10             Cade Mays  CAR   OG   6-6  325   4/26/1999   25   3
11              Cam Gill  CAR   LB   6-3  240  12/14/1997   26   5
12              Cam Sims  CAR   WR   6-5  214    1/6/1996   28   6
13       Chandler Wooten  CAR   LB   6-3  230   4/25/1999   25

Request succeeded with status code 200
                   Name Team  POS    HT   WT         DOB  AGE EXP
0           Aaron Casey  CIN   LB   6-1  231   6/12/2000   24   R
1   Akeem Davis-Gaither  CIN   LB   6-2  228   9/21/1997   26   5
2            Alex Cappa  CIN   OG   6-6  310   1/27/1995   29   7
3          Allan George  CIN   CB  5-11  191   7/16/1999   25   1
4          Amarius Mims  CIN   OT   6-7  340  10/14/2002   21   R
5        Andrei Iosivas  CIN   WR   6-3  205  10/15/1999   24   2
6       Austin McNamara  CIN    P   6-5  204   2/24/2001   23   R
7             B.J. Hill  CIN   DT   6-3  311   4/20/1995   29   7
8          Brad Robbins  CIN    P   6-1  205   10/6/1998   25   2
9          Cal Adomitis  CIN   LS   6-2  238    7/9/1998   26   3
10           Cam Grandy  CIN   TE   6-5  253  11/19/1999   24   R
11     Cam Taylor-Britt  CIN   CB  5-11  202  10/15/1999   24   3
12       Cameron Sample  CIN   DE   6-3  272   9/20/1999   24   4
13       Cedric Johnson  CIN   DE   6

Request succeeded with status code 200
                    Name Team  POS    HT   WT         DOB  AGE EXP
0            Alec Holler  DAL   TE   6-3  242           —   22   R
1          Asim Richards  DAL   OT   6-4  307   10/2/2000   23   2
2          Brandin Cooks  DAL   WR  5-10  189   9/25/1993   30  11
3         Brandon Aubrey  DAL    K   6-3  208   3/14/1995   29   2
4      Brevyn Spann-Ford  DAL   TE   6-7  270  12/27/1999   24   R
5          Brock Hoffman  DAL    C   6-4  302    7/2/1999   25   2
6         Brock Mogensen  DAL   LB   6-2  235           —   22   R
7            Bryan Anger  DAL    P   6-3  202   10/6/1988   35  13
8          Buddy Johnson  DAL   LB   6-2  240   2/27/1999   25   3
9          Byron Vaughns  DAL   LB   6-3  225           —   22   R
10          C.J. Goodwin  DAL   DB   6-3  190    2/4/1990   34   9
11         Caelen Carson  DAL   DB   6-0  195    3/3/2002   22   R
12           Cam Johnson  DAL   WR   6-0  202           —   22   R
13            Carl Davi

...

Request succeeded with status code 200
                     Name Team  POS    HT   WT         DOB  AGE EXP
0              A.T. Perry   NO   WR   6-5  205  10/26/1999   24   2
1           Adam Prentice   NO   RB   6-0  245   1/16/1997   27   4
2          Alontae Taylor   NO   CB   6-1  199   12/3/1998   25   3
3            Alvin Kamara   NO   RB  5-10  215   7/25/1995   28   8
4           Anfernee Orji   NO   LB   6-2  230   10/6/2000   23   1
5             Blake Grupe   NO    K   5-7  156   11/5/1998   25   2
6            Bryan Bresee   NO   DT   6-5  305   10/6/2001   22   2
7               Bub Means   NO   WR   6-2  215   1/10/2001   23   R
8          Cameron Jordan   NO   DE   6-4  287   7/10/1989   35  14
9         Carl Granderson   NO   DE   6-5  261  12/18/1996   27   6
10     Cedrick Wilson Jr.   NO   WR   6-2  197  11/20/1995   28   7
11             Cesar Ruiz   NO   OG   6-4  316   6/14/1999   25   5
12          Charlie Smyth   NO    K   6-4  210   6/26/2001   23   R
13       

Request succeeded with status code 200
                        Name Team  POS    HT   WT         DOB  AGE EXP
0              Aaron Rodgers  NYJ   QB   6-2  223   12/2/1983   40  20
1              Al Blades Jr.  NYJ  SAF   6-1  195  10/24/1999   24   R
2               Allen Lazard  NYJ   WR   6-5  227  12/11/1995   28   6
3             Andrew Peasley  NYJ   QB   6-2  218    1/6/2000   24   R
4               Ashtyn Davis  NYJ  SAF   6-1  202  10/10/1996   27   5
5             Austin Deculus  NYJ   OT   6-5  321   3/12/1999   25   3
6             Austin Seibert  NYJ    K   5-9  200  11/15/1996   27   4
7               Brady Latham  NYJ   OG   6-5  312   9/30/2000   23   R
8              Braelon Allen  NYJ   RB   6-1  235   1/20/2004   20   R
9           Braiden McGregor  NYJ   DE   6-6  267   7/12/2001   23   R
10            Brandin Echols  NYJ   CB  5-10  179  10/16/1997   26   4
11        Brandon Codrington  NYJ   DB   5-9  185           —   22   R
12               Breece Hall  NYJ   RB

Request succeeded with status code 200
                     Name Team  POS    HT   WT         DOB  AGE EXP
0         Aaron Shampklin  PIT   RB  5-10  195  10/18/1999   24   1
1          Alex Highsmith  PIT   LB   6-4  242    8/7/1997   26   5
2          Anderson Hardy  PIT    C   6-6  301    2/3/2000   24   1
3         Anthony Averett  PIT   CB  5-11  184  11/30/1994   29   6
4       Beanie Bishop Jr.  PIT   CB   5-9  182  12/17/1999   24   R
5          Breiden Fehoko  PIT   DT   6-3  300  10/15/1996   27   4
6         Broderick Jones  PIT   OT   6-5  311   5/16/2001   23   2
7       Calvin Austin III  PIT   WR   5-9  162   3/24/1999   25   3
8         Cameron Heyward  PIT   DT   6-5  295    5/6/1989   35  14
9        Cameron Johnston  PIT    P  5-11  194   2/24/1992   32   7
10          Chris Boswell  PIT    K   6-2  185   3/16/1991   33  10
11        Christian Kuntz  PIT   LS   6-1  227   4/13/1994   30   4
12           Cole Holcomb  PIT   LB   6-1  240   7/30/1996   27   6
13       

Request succeeded with status code 200
                    Name Team  POS    HT   WT         DOB  AGE EXP
0              AJ Barner  SEA   TE   6-6  251    5/3/2002   22   R
1          Abraham Lucas  SEA   OT   6-6  322  10/15/1998   25   3
2        Andrew Whitaker  SEA   CB   6-0  183   1/16/1999   25   2
3       Anthony Bradford  SEA   OG   6-5  332   4/28/2001   23   2
4            Artie Burns  SEA   CB   6-0  187    5/1/1995   29   9
5              Boye Mafe  SEA   LB   6-4  261  11/30/1998   25   3
6          Brady Russell  SEA   TE   6-3  250   8/31/1998   25   2
7           Buddha Jones  SEA   NT   6-3  317    2/6/2001   23   R
8        Byron Murphy II  SEA   DT   6-0  306    9/8/2002   21   R
9          Cameron Young  SEA   DT   6-3  304    6/8/2000   24   2
10       Carlton Johnson  SEA   DB  5-11  175           —   22   R
11         Charles Cross  SEA   OT   6-5  311  11/25/2000   23   3
12           Chris Stoll  SEA   LS   6-2  255    8/3/1998   25   2
13      Christian Hayne

Request succeeded with status code 200
                     Name Team  POS    HT   WT         DOB  AGE EXP
0            Amani Hooker  TEN  SAF  5-11  210   6/14/1998   26   6
1          Andrew Rupcich  TEN   OT   6-6  318    4/1/1999   25   1
2         Anthony Kendall  TEN   CB  5-10  188  12/17/1999   24   2
3               Arden Key  TEN   LB   6-5  240    5/3/1996   28   7
4        Brayden Narveson  TEN    K  5-11  215           —   22   R
5            Brian Dooley  TEN   OL   6-5  318           —   22   R
6            Bryce Oliver  TEN   WR   6-1  214           —   22   R
7            Caleb Farley  TEN   CB   6-2  197   11/2/1998   25   4
8            Caleb Murphy  TEN   LB   6-3  254  12/15/1999   24   2
9           Calvin Ridley  TEN   WR   6-1  190  12/20/1994   29   6
10            Cedric Gray  TEN   LB   6-2  235  10/30/2002   21   R
11        Chance Campbell  TEN   LB   6-2  232   10/8/1999   24   3
12         Chidobe Awuzie  TEN   CB  5-11  202   5/24/1995   29   8
13      C